## 19. 동적 속성과 프로퍼티
### 19.1 동석 속성을 이용한 데이터 랭글링
다음에 나올 몇 가지 예제에서는 OSCON 2014 콘퍼런스에서 오라일리가 공개한 JSON 데이터 피드를 사용하기 위해 동적 속성을 이용한다.

In [2]:
""" [예제 19-2] osconfeed.py: osconfeed.json 내려받기 """

from urllib.request import urlopen
import warnings
import os 
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) # 새로 내려받아야 하는 경우 경고 메시지를 출력
        with urlopen(URL) as remote, open(JSON, 'wb') as local: # 두 개의 콘텍스트 관리자를 이용해서 원격 파일을 읽고 저장하는 with 문
            local.write(remote.read())
            
    with open(JSON) as fp:
        return json.load(fp) # JSON 파일을 파싱하고 네이티브 파이썬 객체로 반환하며, 이 피드에는 dict, list, str, int 형의 데이터가 있음

In [3]:
feed = load()
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [4]:
for key, value in sorted(feed['Schedule'].items()):
    print("{:3} {}".format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [5]:
feed['Schedule']['speakers'][-1]['name'] # 마지막 발표자의 이름

'Carina C. Zona'

### 19.1.1 동적 속성을 이용해서 JSON과 유사한 데이터 둘러보기
[예제 19-2]는 아주 간단하지만 feed['Schedule']['event'][40]['name']과 같은 구문은 번거롭다. 예제 19-5의 FrozenJSON을 사용하면 자바스크립트처럼 feed.Schedule.event[40].name과 같은 구문으로 동일한 값을 가져올 수 있다. 

FrozenJSON의 핵심은 \_\_getattr\_\_( ) 메서드다.  \_\_getattr\_\_( ) 특별 메서드는 속성을 가져오기 위한 일반적인 과정이 실패할 때(즉, 지명한 속성을 객체, 클래스, 슈퍼클래스에서 찾을 수 없을 때)만 인터프리터에서 호출한다.

In [17]:
""" [예제 19-5] explore0.py: JSON 데이터셋을 내포한 FrozenJSON 객체, 리스트, 기본형을 담고 있는 FrozenJSON으로 변환 """

from collections import abc

class FrozenJSON:
    """
    점 표기법을 이용해서 JSON과 유사한 객체를 순회하는 읽기 전용 퍼사드 클래스
    """
    
    def __init__(self, mapping):
        self.__data = dict(mapping) # 딕셔너리 메서드를 사용할 수 있으며, 원본을 변경하지 않는다.
        
    def __getattr__(self, name): # name 속성이 없을 때만 __getattr__() 메서드가 호출된다.
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # __data에 들어 있는 객체가 name 속성을 가지고 있으면 그 속성을 반환한다.
        else:
            return FrozenJSON.build(self.__data[name])  # 그렇지 않으면 self.__data에 name을 키로 사용해서 항목을 가져오고 가져온 항목에
                                                        # FrozenJSON.build()를 호출한 결과를 반환한다.
        
    @classmethod # 일반적으로 대안 생성자로 @classmethod를 사용한다.
    def build(cls, obj):
        if isinstance(obj, abc.Mapping): # obj가 매핑형이면 이 객체로부터 FrozenJSON 객체를 생성한다.
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): # obj가 MutableSequence 형이면 리스트이므로 obj안에 있는 모든 항목에 build() 메서드를 적용해서 생성된 객체들의 리스트를 반환한다.
            return [cls.build(item) for item in obj]
        else:
            return obj # obj가 매핑도 아니고 리스트도 아니면 항목을 그대로 반환한다.

In [21]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [22]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [23]:
for key, value in sorted(feed.Schedule.items()):
    print("{:3} {}".format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [24]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [25]:
talk = feed.Schedule.events[40]
type(talk) # JSON 객체였지만 이제는 FrozenJSON 객체가 되었다.

__main__.FrozenJSON

In [26]:
talk.name

'There *Will* Be Bugs'

In [27]:
talk.speakers

[3471, 5199]

In [28]:
talk.flavor # 없는 속성을 읽으려 시도하면 일반적으로 발생하는 AttributeError 대신 KeyError 예외가 발생한다.

KeyError: 'flavor'

### 19.1.2 잘못된 속성명 문제
FrozenJSON 클래스는 파이썬 키워드가 속성명으로 사용된 경우를 처리하지 못한다. 때문에 전달된 매핑 안의 키가 파이썬 키워드인지 검사하고, 파이썬 키워드인 경우에는 뒤에 \_를 붙여 속성을 읽을 수 있게 만드는 것이 좋다.

In [29]:
""" [예제 19-6] 파이썬 키워드인 속성명에 언더바 붙이기 """

def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
        if keyword.iskeyword(key):
            key += '_'
        self.__data[key] = value

In [30]:
x = FrozenJSON({'2be':'or not'})

In [31]:
x.2be

SyntaxError: invalid syntax (<ipython-input-31-8694215ab5bd>, line 1)

### 19.1.3 \_\_new\_\_( )를 이용한 융통성 있는 객체 생성